<a href="https://colab.research.google.com/github/repro-school/training-fens/blob/main/second_level_GLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Higher-level analyses.

In this notebook, we will be essentially following on from the [previous notebook](https://colab.research.google.com/drive/1dMgMQQddOPPCs7sBOnH9YNFQk7SsWtb-?usp=sharing) on first-level analyses we did a few weeks ago. Here, we will be expanding in several ways:

1. We will be performing higher-level GLM analyses.
2. We will take a look at the issue of *correcting for multiple statistical tests*.
3. We will take a look at *region of interest* analyses.


## Want something to listen to while you work? 🎧

[Here](https://www.youtube.com/watch?v=jfKfPfyJRdk) is a playlist I listened to while preparing this notebook. It never lets me down.


## The data
We will be using the same movie-watching data as in the previous notebook. 
However, this time, we will be using a larger quantity of data:

Specifically:

1. Instead of data from 1 subject, we will be modeling data from **5 subjects**.
2. From each subject there are **two runs of data**. In other words, they were scanned twice. In this case, they watched exactly the same movie each time they were scanned. 

Our goal is to generate a set of group-level results that allow us to generalise our results to the population, as is illustrated below.

<img src = 'https://docs.google.com/drawings/d/e/2PACX-1vQxCH3WU3nTqFlHUZb49rf9zioivGQ-flVfRpwmXQx7OF5Wm_1T6gFMYQqpqt-NPITNHUaRoVYEREgT/pub?w=965&h=745' width = "600" height = "" >




## Getting started


First however, in order to install the software that renders our visualisations and all the relevant packages, we need to run the same, rather time-consuming cell that we ran at the start of the last notebook 😴 .

Run the below cell and make yourself a cup of tea/ coffee while you wait. Again, you will see a green tick when it is all finished. This cell shouldnt take more than a few minutes.

In [ ]:
#@title ↓ --- Run this cell by pressing the play button below
# this cell installs some dependencies. 
# feel free to disregard the output this generates

!apt -qq install inkscape &> /dev/null
!pip -qq install nibabel nilearn &> /dev/null
!pip -qq install git+https://github.com/gallantlab/pycortex.git#egg=pycortex &> /dev/null
! pip install googledrivedownloader &> /dev/null
! pip install mne

from platform import python_version
pversion='.'.join(python_version().split('.') [:-1])


import numpy as np
import scipy as sp
import nibabel as nb
import nilearn as nl
from nilearn.surface import load_surf_data
import os, shutil, urllib.request

from matplotlib import rc
from matplotlib import cm
import matplotlib as mpl
rc('animation', html='jshtml')

import matplotlib.pyplot as plt
%matplotlib inline 

import pickle

#
# this cell ensures that we can work with our own surface from within the colab environment
#
wrong_filestore_location='/usr/share/pycortex/db'

os.makedirs('/content/pycortex/db', exist_ok=True)
os.makedirs('/content/pycortex/colormaps', exist_ok=True)
os.makedirs('/content/data', exist_ok=True)

with open('/usr/local/lib/python{pversion}/dist-packages/cortex/defaults.cfg'.format(pversion=pversion), 'r') as f:
  file_source = f.read()

replace_string = file_source.replace(wrong_filestore_location, '/content/pycortex/db') #save output 
with open('/usr/local/lib/python{pversion}/dist-packages/cortex/defaults.cfg'.format(pversion=pversion), 'w') as f:
  f.write(replace_string)   

os.chdir('/tmp/')
!git clone https://github.com/gallantlab/pycortex.git
!cp /tmp/pycortex/filestore/colormaps/* /content/pycortex/colormaps/

#
# and we'll download our average hcp subject for pycortex visualization
#

pycortex_sj_URL = "https://ndownloader.figshare.com/files/25768841"

urllib.request.urlretrieve(pycortex_sj_URL, os.path.join('/content/pycortex/db', 'hcp_999999.zip'))
!unzip -qq /content/pycortex/db/hcp_999999.zip -d /content/pycortex/db/


import numpy as np
import matplotlib.pyplot as plt

print('Everything all set up!')

!pip install --upgrade --no-cache-dir gdown

import gdown
url3 = 'https://drive.google.com/drive/folders/1_LwMr0mphubfVupdYUBk800YoM8yuqTY?usp=sharing&confirm=t'
output3 = '/content/data/HCP-MMP'
gdown.download_folder(url=url3, output=output3, quiet=False)


import nibabel as nib
import numpy as np

class MMP_masker:
    def __init__(self,MMPloc):
        self.MMPloc=MMPloc
        self.load()
        
    def load(self):    
        self.annotfile_L = os.path.join(self.MMPloc,'lh.HCP-MMP1.annot')
        self.annotfile_R = os.path.join(self.MMPloc,'rh.HCP-MMP1.annot')
        
        self.lh_labels, self.lh_ctab, self.lh_names = nib.freesurfer.io.read_annot(self.annotfile_L)
        self.rh_labels, self.rh_ctab, self.rh_names = nib.freesurfer.io.read_annot(self.annotfile_R)
        self.lh_names=self.decode_list(self.lh_names)
        self.rh_names=self.decode_list(self.rh_names)
        
    def decode_list(self,inlist):
        outlist=[x.decode() for x in inlist]
        return outlist
        
    def get_roi_index(self,label,hem='L'):
        
        if hem=='L':
            n2search=self.lh_names
        elif hem=='R':
            n2search=self.rh_names
            
        idx=n2search.index('{hem}_{label}_ROI'.format(label=label,hem=hem))
        
        return idx
    
    def get_roi_verts(self,label):
        Lverts,Rverts=np.where(self.lh_labels==self.get_roi_index(label))[0],np.where(self.rh_labels==self.get_roi_index(label,hem='R'))[0]
        return Lverts, Rverts
    
    def downsample(self,inarray,vertsperhem=10242):
        
        outarray=inarray[:vertsperhem]
        return outarray
        
    def make_roi_mask(self,label,downsample=False,boolean=True):
        L_empty,R_empty=np.zeros(len(self.lh_labels)),np.zeros(len(self.rh_labels))
        Lverts,Rverts=self.get_roi_verts(label)
        L_empty[Lverts]=1
        R_empty[Rverts]=1
        
        if downsample==True:
            L_empty,R_empty=self.downsample(L_empty),self.downsample(R_empty)
        
        combined_mask=np.concatenate([L_empty,R_empty])
        
        if boolean==True:
             L_empty,R_empty,combined_mask=L_empty.astype(bool),R_empty.astype(bool),combined_mask.astype(bool)
            
        return L_empty, R_empty, combined_mask
    
    def make_composite_mask(self,labels):
        roimasks=np.sum([self.make_roi_mask(label) for label in labels],axis=0)        
        return roimasks


## Inspecting the data

Now let's download and inspect the data.

In [ ]:
url1 = 'https://drive.google.com/uc?id=1_KiLfTnxEKr_Geh6r0fIGzQgKZfYGqSY&confirm=t'
output1 = '/content/data/run1_data.npz'
gdown.download(url1, output1, quiet=False)

url2 = 'https://drive.google.com/uc?id=1_Lg_HkdLdW1CWxEGXQOjO-UHaHXMUHmo&confirm=t'
output2 = '/content/data/run2_data.npz'
gdown.download(url2, output2, quiet=False)


import numpy as np
ts_data1=np.load('/content/data/run1_data.npz',allow_pickle=True)
ts_data2=np.load('/content/data/run2_data.npz',allow_pickle=True)


In [ ]:
run_1_data=np.array(ts_data1['arr_0'])
run_2_data=np.array(ts_data2['arr_0'])
run_1_data.shape, run_2_data.shape

In [ ]:
run_1_data.size + run_2_data.size

Yikes, thats a lot of data 😨. 243097200 data points in total !!

&nbsp;

Let's break this down:


Our data conist of two arrays of size (5, 118584, 205)

As we know from the last notebook the *118584* refers to the number of voxels and *205* refers to the number of timepoints.

We have *2 arrays* because there are two functional runs for each participant.

The *5*, then, corresponds to the number of participants in our dataset.

As in the previous notebook, lets plot the time-averaged response onto the brain. This time, lets do it for each subject. We will plot the first run only to avoid cluttering the figure.

In [ ]:
import cortex
subject_means=np.mean(run_1_data,axis=2)
f, s = plt.subplots(2,3,figsize=(24,10))
f.suptitle('Subject mean responses for run 1')


for c,v in enumerate(s.reshape(-1)):
  if c==5:
    break
  vas_rgb_v=cortex.Vertex(subject_means[c], subject='hcp_999999', cmap='cubehelix',vmin=np.nanmin(subject_means[c]),vmax=np.nanmax(subject_means[c])) 
  v.title.set_text("Subject {sub}".format(sub=str(c+1)))

  cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,fig=v);


plt.tight_layout()

Here, although the general profile is the same (large response in visual  and auditory cortices), we can actually see quite a bit of individual subject variation in the mean BOLD response to the movie. 


To illustrate the individual variability in responses, here I take the mean data within V1 and plot it for each of the 5 subjects, which appear as seperate lines. 

In [ ]:
V1_data=np.mean(run_1_data[:,cortex.utils.get_roi_verts('hcp_999999','V1')['V1']],axis=1)
from matplotlib.pyplot import figure

figure(figsize=(12, 3), dpi=80)
plt.title('Mean V1 reponses for each subject in run 1')


plt.plot(V1_data.T)

Here we see that even though subjects are watching exactly the same movie, the responses in visual cortex are quite fundamentally different from one another. 

Let's see what the between-subject correlations are for mean V1 timecourses for each participant.

In [ ]:
np.corrcoef(V1_data)

Wow, some of the correlations are as low as .03. There are a number of reasons why we might see such divergent brain responses.

<img src = 'https://dragonflytraining.files.wordpress.com/2013/10/man-with-question-01.png?w=600&h=600' width = "100" height = "" >

**QUESTION:** What might be some of the sources of this between-subject variability in responses?

Now lets look at the mean response in A1 (primary auditory cortex) for each subject

In [ ]:
A1_data=np.mean(run_1_data[:,cortex.utils.get_roi_verts('hcp_999999','A1')['A1']],axis=1)
from matplotlib.pyplot import figure

figure(figsize=(12, 3), dpi=80)

plt.plot(A1_data.T)



<img src = 'https://dragonflytraining.files.wordpress.com/2013/10/man-with-question-01.png?w=600&h=600' width = "100" height = "" >



**QUESTION:** In both V1 and A1 we can see that all subjects have a large, elevated response at the very start of the movie. What do you think might be going on here?


# Take a break 1.

Ok, lets take a break and discuss what we have done so far.

<img src = 'https://upload.wikimedia.org/wikipedia/commons/8/81/Stop_sign.png' width = "200" height = "" >


# Modeling our data

## Combining across runs - a fixed effects model.

Ok, next we want to try and model our individual subject data. To make this simple, let's only interest ourselves in one effect, which is a simple contrast between two ${\beta}$ (or  $c\hat{\beta}$).

 Let's say we are interested in finding regions that have a signficantly larger response to bodies than to scenes.

 In other words, testing the null hypothesis:

&nbsp;


${\beta}$  (bodies) - ${\beta}$  (scenes) ==0

&nbsp;

First of all, let's make our design matrix (X in the GLM equation). This is no different from our last notebook - so if you want to review the stages that we went through to generate that - then take a look at the other notebook.

For now, I have hidden the code in the below cell, since it is rather a lot.

In [ ]:
#@title ↓ --- Run this cell by pressing the play button below
# this cell installs some dependencies. 
# feel free to disregard the output this generates
body_starts=[26, 49, 61, 72, 88, 93, 98, 104, 116, 139, 156, 164, 170]
body_ends=[33, 57, 71, 75, 90, 95, 99, 111, 136, 140, 158, 166, 190]
text_starts=[0, 21, 27, 74, 83, 91, 107, 150, 137]
text_ends=[5, 25, 31, 78, 89, 95, 109, 152, 140]
scene_starts=[20, 34, 79, 112, 146, 153, 159, 166]
scene_ends=[21, 36, 82, 116, 150, 156, 162, 170]


def make_boxcar_from_times(secsbeg,secsend,full_length=205):

    event=np.zeros(full_length)      

    for c,v in enumerate(secsbeg):
        event[secsbeg[c]:secsend[c]]=1 
        
    return event

body_event=make_boxcar_from_times(body_starts,body_ends)
scene_event=make_boxcar_from_times(scene_starts,scene_ends)
text_event=make_boxcar_from_times(text_starts,text_ends)
events=np.vstack([body_event,scene_event,text_event])

from matplotlib.pyplot import figure


from scipy import signal
from nilearn.glm.first_level.hemodynamic_models import spm_hrf, spm_time_derivative, spm_dispersion_derivative

def create_hrf(hrf_params=[1.0, 1.0, 0.0],TR=1):
    """
        
    construct single or multiple HRFs        
    Parameters
    ----------
    hrf_params : TYPE, optional
    DESCRIPTION. The default is [1.0, 1.0, 0.0].
    Returns
    -------
    hrf : ndarray
    the hrf.
    
    
    Adapted from prfpy
    """
        
    hrf = np.array([np.ones_like(hrf_params[1])*hrf_params[0] *spm_hrf(tr=TR,oversampling=1,time_length=40)[...,np.newaxis],
    hrf_params[1] *spm_time_derivative(tr=TR,oversampling=1,time_length=40)[...,np.newaxis],hrf_params[2] *
    spm_dispersion_derivative(tr=TR,oversampling=1,time_length=40)[...,np.newaxis]]).sum(axis=0)
    

    return hrf.T

hrf=create_hrf()


def convolve(hrf,regressor):
    
    """
    Performs standard HRF convolution with a regressor.     
    Parameters
    ----------
    hrf : a HRF (i.e. created by create_hrf)
    Regressor: A fmri timecourse regressor
    Returns
    -------
    conv : The convolve regressor.
    """
    
    
    
    hrf_shape = np.ones(len(regressor.shape), dtype=np.int)
    hrf_shape[-1] = hrf.shape[-1]
    conv=signal.fftconvolve(regressor,hrf.reshape(hrf_shape), mode='full', axes=(-1))[..., :regressor.shape[-1]]
    return conv

regs=convolve(hrf,events)
import pandas as pd
nldm=pd.DataFrame(regs.T,columns=['body','scene','text'])
nldm=nldm.assign(intercept=1)

X=nldm.values

Ok, now we have generated our design matrix. Remember, the columns are

1. bodies
2. scenes
3. text
4. and the intercept.

In [ ]:
plt.matshow(X, fignum=1, aspect='auto')

Take another look at the figure below. The first thing that we need to do is grapple with the fact that we have two runs per subject. We can use a *fixed-effects model* (FFX) to get a combined estimate of our effect across runs. This allows us to generate one effect per subject that we can then pass up to our group analysis. 

&nbsp;


<img src = 'https://docs.google.com/drawings/d/e/2PACX-1vQxCH3WU3nTqFlHUZb49rf9zioivGQ-flVfRpwmXQx7OF5Wm_1T6gFMYQqpqt-NPITNHUaRoVYEREgT/pub?w=965&h=745' width = "600" height = "" >




As I mentioned in the lecture, this is essentially akin to concatenating the Y (data) and X (design matrix) for each run into one 'long' X and Y.

In other words, we model the data as if this is all emanated from one long run - and we essentially completely ignore the fact that there are two different runs in our modeling. Let's concatenate the data from each run into one long dataset with 410 TRs instead of 205.



 

In [ ]:
full_data=np.concatenate([run_1_data,run_2_data],axis=2)

In [ ]:
full_data.shape

Thus, we now have a 410 TR timeseries for each of the 5 subjects.

For instance, here is what our new mean V1 data looks like. I have drawn a blue line that illustrates where one run ended and the other began. 

In [ ]:
figure(figsize=(15, 8), dpi=80)
V1_data_long=np.mean(full_data[:,cortex.utils.get_roi_verts('hcp_999999','V1')['V1']],axis=1)
from matplotlib.pyplot import figure

figure(figsize=(12, 3), dpi=80)
plt.title('Mean V1 reponses for each subject concatenated across runs')
plt.text(x=100, y=1.2,s='Run 1')
plt.text(x=300, y=1.2,s= 'Run 2')

plt.plot(V1_data_long.T)
plt.axvline(x=205,linewidth=4, color='b')


Note that now we have concatenated our data from each run (*Y*) into one timeseries of 410 TRs, in order to fit the model, we will also need to do the same for *X* -  or our design matrix.

Since the subjects watched the same movie in each run, the timing of events will be no different. Therefore, this basically just involves duplicating our original matrix and stacking the two design matrices on top of each other. 

In [ ]:
X_long=np.tile(X.T,2).T
plt.matshow(X_long, fignum=1, aspect='auto')

Now all that remains is to perform a GLM on this concatenated data for each subject. This will give us one contrast image per subject - thereby effectively combing our data across runs.

Since we already went through all the nuts and bolts of GLM modeling in our previous notebook it is now trivial for us to go through each one of these subjects, solve the GLM equation for each one and compute the body>scene contrast for each subject.

&nbsp;

Below I define a function that takes the design matrix (X), the data (Y) and a contrast vector to compute:
1. the numerator of the t statistic ($c\hat{\beta}$) (effect or cope in FSL terminology).
2. The denominator of the t statistic (uncertainty, or varcope in FSL terminology).
3. and the t statistic itself (effect/ uncertainty) for a given contrast.

If you read the code below, youll notice this is all **exactly** the same things we did in the last notebook for our single subject analyses. The only difference is that now we are now doing this for each subject in our dataset.

I have annotated the code below to refresh your memory of what we are doing

In [ ]:
import numpy.linalg as npl

def compute_contrast(X,Y,contrast=[1,-1,0,0]): # Our contrast is bodies v scenes (this essentially subtracts the scene column from the body column)
  contrast1=np.array(contrast)
  Y=np.nan_to_num(Y.T)


  # GLM
  beta=np.linalg.solve(X.T @ X, X.T @ Y) # Solve the GLM equation, estimating beta.

  # Contrast/effect/cope (numerator of t statistic)
  t_numerator=np.dot(contrast1,beta) # The contrast image is the dot product of the contrast vector and estimated betas.
  
  # Uncertainty/ varcope stuff
  n = Y.shape[0] # Timepoints
  df_error = n - npl.matrix_rank(X) # Timepoints- precictors.
  print(df_error)
  yhat=np.dot(X,beta) # Model predictions.
  E = Y - yhat # Error
  sigma_2 = np.sum(E ** 2, axis=0) / df_error # Sum of squared errors

  design_variance= contrast1.dot(npl.pinv(X.T.dot(X))).dot(contrast1) # Design variance
  t_denominator=np.sqrt(sigma_2 * design_variance) # T denominator

  # Compute t as the ratio of effect and uncertainty.
  tstat=t_numerator/t_denominator


  return t_numerator,t_denominator,tstat,sigma_2 

Ok, lets do all the fitting and contrast estimation for each subject in a *loop*.

 In other words, below we apply the function above to all 5 subjects in our full dataset and return the results in a list.

Hence, we will get a list, with each of the 5 entries in the list containing the results for each subject.

In [ ]:
results_fixed_effects=[compute_contrast(X_long,v) for c,v in enumerate(full_data)]

Great, now we can plot the contrast ($c\hat{\beta}$) images for ${\beta}$  (bodies) - ${\beta}$  (scenes) that we estimated for each subject. Here, brighter regions indicate a larger difference in the response to ${\beta}$  (bodies) - ${\beta}$ (scenes) 


In [ ]:
f, s = plt.subplots(2,3,figsize=(24,10))
f.suptitle(r'$\beta$'+ ' body -' r' $\beta$'+ ' house' +'contrast image for each subject estimated by fixed-effects modeling across runs')


for c,v in enumerate(s.reshape(-1)):
  if c==5:
    break
  betahat=results_fixed_effects[c][0]
  vas_rgb_v=cortex.Vertex(betahat, subject='hcp_999999', cmap='cubehelix',vmin=np.nanmin(betahat),vmax=np.nanmax(betahat)) 
  v.title.set_text("Subject {sub}".format(sub=str(c+1)))

  cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,fig=v);


plt.tight_layout()

Cool. Keep in mind that each subject also has the numerator for their t statistic (or VARCOPE) which we can also plot. Here, brighter regions indicate greater uncertainty.

In [ ]:
f, s = plt.subplots(2,3,figsize=(24,10))
f.suptitle('Uncertainty for each subject')


for c,v in enumerate(s.reshape(-1)):
  if c==5:
    break
  varcopes=results_fixed_effects[c][1]
  vas_rgb_v=cortex.Vertex(varcopes, subject='hcp_999999', cmap='cubehelix',vmin=0.15,vmax=0.31) 
  v.title.set_text("Subject {sub}".format(sub=str(c+1)))

  cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,fig=v);


plt.tight_layout()

And of course, we have the t-statistics themselves. Here I plot negative t statistics (𝛽  (scenes) -  𝛽  (bodies)) in blue and positive t statistics (𝛽  (bodies) -  𝛽  (scenes))  in red. 

In [ ]:
f, s = plt.subplots(2,3,figsize=(24,10))
f.suptitle('Subject t statistics responses')


for c,v in enumerate(s.reshape(-1)):
  if c==5:
    break
  tstat=results_fixed_effects[c][2]
  vas_rgb_v=cortex.Vertex(tstat, subject='hcp_999999', cmap='BuWtRd',vmin=-2,vmax=2) 
  v.title.set_text("Subject {sub}".format(sub=str(c+1)))

  cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,fig=v);


plt.tight_layout()

Let's threshold the t statistic above 2 so that we can see the regions that have a significantly larger response to bodies v scenes

In [ ]:
f, s = plt.subplots(2,3,figsize=(24,10))
f.suptitle('Tstat > 2')


for c,v in enumerate(s.reshape(-1)):
  if c==5:
    break
  tstat=results_fixed_effects[c][2]
  tstat[tstat<2]=np.nan
  vas_rgb_v=cortex.Vertex(tstat, subject='hcp_999999', cmap='plasma',vmin=2,vmax=5) 
  v.title.set_text("Subject {sub}".format(sub=str(c+1)))

  cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=True,fig=v);


plt.tight_layout()

There is actually a tremendous amount of consistency here. Allow me to illustrate this by zooming into a portion of lateral visual cortex in the right hemisphere (it's more or less for the left hemisphere).



In [ ]:
zoomrect=[70,160,-90,40]

def zoom_to_rect(myrect):
    plt.axis(myrect)

f, s = plt.subplots(2,3,figsize=(15,10))
f.suptitle('Tstat > 2 for lateral cortex - right hemisphere')


for c,v in enumerate(s.reshape(-1)):
  if c==5:
    break
  tstat=results_fixed_effects[c][2]
  tstat[tstat<2]=np.nan
  vas_rgb_v=cortex.Vertex(tstat, subject='hcp_999999', cmap='plasma',vmin=2,vmax=5) 
  v.title.set_text("Subject {sub}".format(sub=str(c+1)))

  cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=True,fig=v);
  zoom_to_rect(zoomrect)




You see what I mean about consistency?

&nbsp;

You should be able to make out 2 different clusters here. The upper right one (upper in this flatmap is more lateral in the brain) is generally larger and more rounded. The lower one (lower in this flatmap is more ventral in the brain) is more elongated and goes in a upper left to lower right diagonal direction.

For my money, the former cluster is the extrastriate body area (EBA) and the latter is the fusiform body area (FBA).

<img src = 'https://www.frontiersin.org/files/Articles/13143/fnhum-05-00124-r2/image_m/fnhum-05-00124-g001.jpg' width = "300" height = "" >



These are canonical body-selective regions that are known for their selectivity to visual presentations of bodies above and beyond other categories of stimuli. You can read a little background on these regions [in this review paper](https://d1wqtxts1xzle7.cloudfront.net/42308990/Kroppsperception-with-cover-page-v2.pdf?Expires=1669061060&Signature=X29ebbEn-fIMHSwCggAd3IZHW38CHt8Cvyu8rICtOU32zfUYJedBaZlMI05M3yZXIXL58IifwujWuQugkLj-e9Vjj56YOPG6bVO1rYi4Byn2vr7cbTjkFkwRqxxybJJFq6KHG6-vFjOpcuBu4PDOxj4ZMgU~7ZGYaThPEQTgJa36tXDyx~myVpaGiFFou~W7oS-Bs2KT8vQRhEbODec2hs0fcD764iA34i-fC1wL8dWI6Ad8WvmDm-6QTLo0bdbn9zjQfxpPdMuEys5nn0PsxXxXIE-cO-G5H47O13Ewm-iVn7yaGQq4xFt68q8OhSb6X2gxtDjyCh98uubuztRTVg__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA). But trust me, this is what we are looking at. 😀 

&nbsp;

Lets now try and visualise the overlap of these significant regions for each subject. We can do this by:

1. Creating a binary mask that is 1 for tstat >2 and 0 for tstat2 <2 for each subject.
2. Taking the sum of these masks.
3. The resulting image therefore reflects the number of subjects that have tstats >2 at each location.

In [ ]:
tstatmasks=np.array([results_fixed_effects[c][2]>2 for c,v in enumerate(results_fixed_effects)])
summed_mask=np.sum(tstatmasks,axis=0)

In [ ]:
vas_rgb_v=cortex.Vertex(summed_mask, subject='hcp_999999', cmap='gist_ncar',vmin=0,vmax=5) 

cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=True);
zoom_to_rect(zoomrect)

Here, we have a representation of the number of subjects that have a t statistic of >2 at each location. 



1. Dark blue = 0 subjects
2. Light blue/ turquoise = 1 subject
3. Green = 2 subjects
4. Orange = 3 subjects
5. Pink = 4 subjects
6. White = 5 subjects

Hence, we actually see that a lot of this portion of cortex is significant for the majority of subjects.



## Single subject analyses to group analyses.


Critically, what we have looked at so far are a bunch of single subject t-tests. Thus they answer the quite limited question:

&nbsp;

"**What regions of the brain have significantly larger response to bodies than scenes for this subject?**" 

&nbsp;

What we really want, is to combine these lower level statistics into one t statistic that answers something more like:

&nbsp;

""**What regions of the brain have significantly larger response to bodies than scenes in the general population?**""

&nbsp;

In other words, we want to use our sample to make *inferences* about the *population*.

&nbsp;

<img src = 'https://online.stat.psu.edu/public/stat800/lesson04/InferenceGraphicSU17.png' width = "300" height = "" >


This is not something that we can do with a fixed effects model. Fixed effects modeling is legitimate for combining data across runs - however it is ill-suited for making inferences like this. 

Thus, In order to answer the latter question, we need to model the inter-subject variability in this effect with a *random-effects (RFX)* model. 


I recomend watch this brief video that goes into the difference between these two types of model.



In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('-abMLQSjMSI')

&nbsp;

<img src = 'https://dragonflytraining.files.wordpress.com/2013/10/man-with-question-01.png?w=600&h=600' width = "100" height = "" >

**QUESTION:** What is your hunch about whether these EBA and FBA activations we looked at will be significant at the Group level? Why?


# Take a break 2.

Ok, lets take a break and discuss what we have done so far.

<img src = 'https://upload.wikimedia.org/wikipedia/commons/8/81/Stop_sign.png' width = "200" height = "" >

## Random effects model: Combining estimates across subjects to generalize to the population.

Ok, let's think about what we have done so far.

We have computed a fixed effects estimate of the single subject contrast ($c\hat{\beta}$) ${\beta}$  (bodies) - ${\beta}$  (scenes). Therefore we have one of these contrasts for each subject.

Let's recap on the GLM equation for a first level model

\begin{align}{y} = X\beta + e\end{align} 



At the first level

1. X is our design matrix.
2. Y is our single-subject data.
3. ${\beta}$ estimates the influence that each of our predictors have on the signal.
4. *e* is the error of our model - the discrepancy between our model predictions and the data.

&nbsp;

At the second level, the GLM equation is the same, but we use different notation to indicate we are now at the group level.

\begin{align}
y^{\dagger} = \mathbf{X}^{\dagger}\beta^{\dagger} + \epsilon^{\dagger}
\end{align}


The results from our run-level analyses ($c\hat{\beta}$) across different subjects will become our target ($y^{\dagger}$). Note that we will use the "dagger" ($^{\dagger}$) superscript to denote that the mathematical terms belong to the group-level model.

&nbsp;


To reiterate, the results from our lower-level analyses ($c\hat{\beta}$) become our dependent variable in our group-level analysis ($y^{\dagger}$):

\begin{align}
y^{\dagger} = c\hat{\beta}
\end{align}

Here is a graphical representation of this. In this example, we see the individual subject contrast ($c\hat{\beta}^{*}$) of ${\beta}$ faces v ${\beta}$ houses becoming an element of $Y^{\dagger}$ for the second level.  


<img src = 'https://dartbrains.org/_images/TwoLevelModel.png' width = "700" height = "" >


Again, the group-level represents a GLM with a particular design matrix ($\mathbf{X}^{\dagger}$) and parameters ($\beta^{\dagger}$):

\begin{align}
y^{\dagger} = \mathbf{X}^{\dagger}\beta^{\dagger} + \epsilon^{\dagger}
\end{align}

And the group-level parameters can be estimated with OLS in exactly the same way as at the first level:

\begin{align}
\hat{\beta}^{\dagger} = (\mathbf{X}^{\dagger\ T} \mathbf{X}^{\dagger})^{-1}\mathbf{X}^{\dagger}y^{\dagger}
\end{align}



Thus, the very first thing that we need to do is construct our ($\mathbf{Y}^{\dagger}$)  from the first-level effects ($c\hat{\beta}$).

Accordingly, below we go through our list of fixed-effects results and bundle our lower $c\hat{\beta}$ for each subject into a matrix to create our($\mathbf{Y}^{\dagger}$).

<img src = 'https://dragonflytraining.files.wordpress.com/2013/10/man-with-question-01.png?w=600&h=600' width = "100" height = "" >

**QUESTION**: Before going ahead, try to guess what shape this matrix will have.

In [ ]:
betas_to_y_dagger=np.array([v[0] for c,v in enumerate(results_fixed_effects)])

In [ ]:
betas_to_y_dagger.shape

Thus, our ($\mathbf{Y}^{\dagger}$) is 5 (subjects) * 118584 (voxels), with each value reflecting the lower level effect ($c\hat{\beta}$).

&nbsp;

Lets think now about our design matrix ($\mathbf{X}^{\dagger}$) . As I mentioed, we want to ask the question:

&nbsp;

""**What regions of the brain have significantly larger response to bodies than scenes in the general population?**""

&nbsp;


Since we have already computed the difference between the body and scene betas at the lower level, this is now a question that is suitable for being adressed via a *one sample t test*. In other words, our null hypothesis is that the overall mean ($c\hat{\beta}$) =0.

&nbsp;

Here is a useful table that details some typical design matrices for second-level analyses:

&nbsp;

<img src = 'https://dartbrains.org/_images/DesignMatrices.png' width = "500" height = "" >


&nbsp;

Look at the difference in complexity between the design matrix for a one sample and paired sample t-test. As you can see, we kind of dodged a bullet here by computing the ($c\hat{\beta}$) at the lower level. 


If we didnt do this, we would have to to include a whole bunch of extra regressors to model out the participant means 😌. We have already done this by essentially carrying a difference score ($c\hat{\beta}$) up to the second level instead of the ${\beta}$ for bodies and scenes seperately.

As such, as you can see here all we have to do is construct a design matrix with one column, that consists of ones for all rows. This models the mean ($c\hat{\beta}$) across subjects. A design matrix for a one sample t-test is as simple as that!





In [ ]:
X_dagger=np.ones((5,1))

In [ ]:
plt.matshow(X_dagger, fignum=1, aspect='auto')

Unlike the first level design matrix, this all appears in the same color because it is just a column of ones. **Simple**

Now, we have ($\mathbf{Y}^{\dagger}$)  and ($\mathbf{X}^{\dagger}$) all we need to do is fit our model to estimate ($\beta^{\dagger}$) - our sample mean.

The cool thing is, since the GLM equation is the same as it was at the first level, we can actually use exactly the same function that we used at the lower level.

The only difference is that we need a different *contrast*.

Here, I define this contrast vector as [1]. Since the dot product of the ($\beta^{\dagger}$) and 1 will just multiply the only column of ($\beta^{\dagger}$) by one - all this does is select out the sample mean that we model. Therefore it barely deserves to be called a contrast, since it is identical to ($\beta^{\dagger}$).

In [ ]:
result_random_effects=compute_contrast(X_dagger,betas_to_y_dagger.T,[1])

I do hope you are as excited as I am to see the group-level results....

Lets take a look at the thresholded t statistics.

In [ ]:
tstat_thresh=np.copy(result_random_effects[2])
tstat_thresh[tstat_thresh<2]=np.nan
vas_rgb_vt=cortex.Vertex(tstat_thresh, subject='hcp_999999', cmap='plasma',vmin=2,vmax=5) 

cortex.quickshow(vas_rgb_vt, with_labels=False, with_rois=False,with_curvature=True);
plt.title("Group level results")


It looks as though we have some significant results at the group level!

Dont get too excited yet though... things are about to get a bit more complicated.

If we look at our t numerator we can see the average effect - or sample mean for our body-scene contrast.

In [ ]:
copes=result_random_effects[0]
vas_rgb_v=cortex.Vertex(copes, subject='hcp_999999', cmap='cubehelix',vmin=np.nanmin(copes),vmax=np.nanmax(copes)) 
cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=True);

If we look at our t denominator, we can see the *between subject variability in effects*. Here, brighter colors indicate regions that have more variable responses across subjects.

&nbsp;

Note that this is different to the fixed effects model, where our denominator reflected the *within-subject uncertainty* in the model's ability to explain the timeseries data, with respect to the *e* of the model and the design variance.

In [ ]:
varcopes=result_random_effects[1]
vas_rgb_v=cortex.Vertex(varcopes, subject='hcp_999999', cmap='cubehelix',vmin=np.nanmin(varcopes),vmax=np.nanmax(varcopes)) 
cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=True);

Now, what about the EBA and FBA regions we looked at earlier? Are these significant at the group level?

In [ ]:
cortex.quickshow(vas_rgb_vt, with_labels=False, with_rois=False,with_curvature=True);
zoom_to_rect(zoomrect)

Indeed they are.

Before we go any further, Id like us to free up some memory by deleting some variables we dont need anymore.

In [ ]:
del ts_data1,ts_data2,run_1_data,run_2_data

Before wrapping up this section, I would like us to take a look at just one voxel, just to make sure that we know what we are estimating.

In [ ]:
vert_number=86820

plt.scatter(np.ones(5), betas_to_y_dagger[:,vert_number], s=120, facecolors='none', edgecolors='r')
plt.ylabel("Single subject effect size")
plt.axhline(y=0,c='k')
plt.axhline(y=copes[vert_number],c='r',label='group mean (cope)')
plt.errorbar(x=1, y=copes[vert_number], yerr=varcopes[vert_number], markersize=8,fmt='o',label='variance (varcope)',uplims=True, lolims=True)
plt.legend()
plt.ylim([-0.5,3])

Here, for one voxel, I have plotted the individual subject estimates of ($c\hat{\beta}$) for each subject (red circles). I then plot the sample mean ($\beta^{\dagger}$), which is what we estimate at the second level as a red line. I also plot the uncertainty of this effect (blue error bar) - which summarises the between subject uncertainty.

&nbsp;

Thus for every voxel in the brain, we have estimated a sample mean and estimate of between-subject uncertainty like this.

&nbsp;

We can see that this particular voxel has high sample mean (far away from zero, where the null hypothesis is true) and not very much variability between subjects. This is because all of the underlying single subject estimates seem to have quite similar estimates of ($c\hat{\beta}$).

Recall the formula for the t statistic:

\begin{align}
t_{\hat{\beta}} = \frac{\hat{\beta}}{\mathrm{SE}_{\hat{\beta}}}
\end{align}


Hence, since the t value is the size of the effect scaled by uncertainty, we should expect this voxel to have a high t value.  

In [ ]:
tstat_thresh[vert_number]

Since the t value is high, we also should expect it to have a low p value too.

In [ ]:
import scipy.stats as stats
t_dist = stats.t(4)
p = 1 - t_dist.cdf(tstat_thresh[vert_number])
p

Thus, we can reject the null hypothesis and say:

**"For this voxel, bodies have a significantly larger influence on its response than scenes in the general population."**

This is exactly the kind of group-level inference we wanted to make. 

&nbsp;

However, unfortunately this gets more complicated......

# Take a break 3.

Ok, lets take a break and discuss what we have done so far.

<img src = 'https://upload.wikimedia.org/wikipedia/commons/8/81/Stop_sign.png' width = "200" height = "" >

## Correction for multiple comparisons.

So far, we have done *no correction for multiple comparisons*. 

&nbsp;

This problem is explained nicely by the below comic.

The comic shows a lab finding a link between acne and jelly beans when a hypothesis was tested at an $\alpha = 0.05$ significance level. Although there is no link between jelly beans and acne, a significant result was found simply by virtue of the fact that the test was conducted multiple times.

&nbsp;

Since the lab tested 20 colors of jelly beans an $\alpha = 0.05$ this led to 1 jelly bean being incorrectly fingered as being the acne culprit (20 tests * .05 = 1 false positve). The implications for false discovery in hypothesis testing is that if you repeat a test enough times, you’re going to find an effect … but that effect may not actually exist.

&nbsp;

<img src = 'https://imgs.xkcd.com/comics/significant.png' width = "700" height = "" >


&nbsp;

This is particularly problematic for our data because there is a test being performed for every voxel in our dataset. If we use a threshold p < 0.05 for each individual test, we would expect many voxels to be declared significant even if there were no true effect. In other words, we would make many type I errors.

&nbsp;


To see why, consider a standard $\alpha = 0.05$.
For a single test, our probability of making a type I error is 0.05.
The probability of making at least one type I error in
$N_{\mathrm{test}}$ independent tests is then given by
$1 - (1 - \alpha)^{N_{\mathrm{test}}}$

Thus, as show below, even if we conduct just 100 tests, we are virtually guaranteed at least one false positive. The scary thing is that we are performing 118584 tests....

In [ ]:
N = np.linspace(1, 100,100)
alpha = 0.05
p_type_I = 1 - (1 - alpha) ** N
fig, ax = plt.subplots(figsize=(4, 3))
ax.scatter(N, p_type_I, 3)
ax.set(xlim=N[[0, -1]], ylim=[0, 1], xlabel=r'$N_{\mathrm{test}}$',
       ylabel=u'Probability of at least\none type I error')
ax.grid(True)
fig.tight_layout()
fig.show()

### Bonferroni correction
Perhaps the simplest way to deal with multiple comparisons, the [Bonferroni
correction](https://en.wikipedia.org/wiki/Bonferroni_correction)
conservatively divides our alpha level by the number of tests. Thus, in our case, we would have to reach the much smaller *p* value of 0.00024 (.05/118584) to reject the null. Let's apply this correction and see which of our voxels values meet this new threshold. 

In [ ]:
pvals = 1 - t_dist.cdf(result_random_effects[2])
data_to_plot=cortex.Vertex2D(result_random_effects[2],pvals<(.05/betas_to_y_dagger.shape[1]),vmin=2,vmax=5,vmin2=0,vmax2=1,subject='hcp_999999',cmap='plasma_alpha')
flatmap=cortex.quickshow(data_to_plot,with_curvature=True,with_rois=False)

.......

<img src = 'https://i.pinimg.com/736x/e8/c5/ac/e8c5aceeedf18ea422eb9a7898f9bc72.jpg' width = "100" height = "" >

The reason this plot is blank is that we are left with no voxels surviving this correction. This really highlights how punitive this correction can be.

<img src = 'https://dragonflytraining.files.wordpress.com/2013/10/man-with-question-01.png?w=600&h=600' width = "100" height = "" >

**QUESTION:** What do you think is the main reason we are no longer getting any significant results?


### FDR correction
As you've seen so far, uncorrected results are probably too relaxed and Bonferroni-corrected results are probably too stingent. [The "False Discovery Rate-correction" (FDR)](https://en.wikipedia.org/wiki/False_discovery_rate#Classification_of_multiple_hypothesis_tests) technique is a method to adjust $p$-values in a way that falls somewhere between the two approaches.

Essentially, while  Bonferroni tries to control the chance of finding at least one false positive result **amongst all your tests**, the FDR-method limits the proportion of false positives **amongst all your tests which turned out significant**. So, if you set your "FDR-proportion" (confusingly also referred to as "alpha") to 0.05, then it will adjust your initial $p$-values such that out of all your significant results, on average 5% will be false positives. You can read more about the simple mathematics of the FDR correction [here](https://wiki.q-researchsoftware.com/wiki/False_Discovery_Rate_Correction) and there is amore detailed, graphical summary[here](https://matthew-brett.github.io/teaching/fdr.html) 

Let's check out what our results look like after FDR correction:

In [ ]:
from statsmodels.stats.multitest import fdrcorrection
alpha_fdr = 0.05

fdr_result = fdrcorrection(np.nan_to_num(pvals,nan=np.nanmean(pvals)), alpha=alpha_fdr)


data_to_plot=cortex.Vertex2D(result_random_effects[2],fdr_result[0],vmin=2,vmax=5,vmin2=0,vmax2=1,subject='hcp_999999',cmap='plasma_alpha')
flatmap=cortex.quickshow(data_to_plot,with_curvature=True,with_rois=False)


<img src = 'https://i.pinimg.com/736x/e8/c5/ac/e8c5aceeedf18ea422eb9a7898f9bc72.jpg' width = "100" height = "" >

We are left with no voxels surviving this correction either.


We can also visualise the corrections that we have made by plotting our corrected p values for each method against the original p value.


In [ ]:
bonf_corrections=pvals*betas_to_y_dagger.shape[1]
bonf_corrections[bonf_corrections>=1]=1

plt.plot(fdr_result[1],pvals,'*',label='FDR correction')
plt.plot(bonf_corrections,pvals,'*',label='bonferroni correction')
plt.xlabel("FDR p value")
plt.ylabel("Orignal p value")
plt.axvline(x=0.05,c='k')
plt.axhline(y=0.05,c='k')

plt.text(x=0.2, y=.7,s='min bonf p value = {bonf}'.format(bonf=np.nanmin(bonf_corrections)))
plt.text(x=0.2, y=.6,s='min FDR p value = {FDR}'.format(FDR=np.nanmin(fdr_result[1])))

plt.legend()

Here we see that a lot of our original p values were significant (values below black horizontal line), but none of our corrected p values are significant (values to the left of the vertical line)


We can see that we get a lot closer to a p value of .05 (black vertical line) with the FDR method. (blue) than with the bonferroni method (orange). In fact, the vast majority of our bonferroni-adjusted p values are 1 (they cannot exceed 1 since this is a probability) - which really highlights how stringent this procedure is.

### Cluster-based correction.

Ok, let's think about this a little differently. Let's take a look at our raw tstats.



In [ ]:
vas_rgb_vt=cortex.Vertex(result_random_effects[2], subject='hcp_999999', cmap='BuWtRd',vmin=-2,vmax=2) 

cortex.quickshow(vas_rgb_vt, with_labels=False, with_rois=False,with_curvature=True);
plt.title("Group level results")

The assumption we have been making so far is that our tests are independent of one another. 

However, do these tests really look independent of one another? We see that high t statistics are adjacent with high t statistics (red) and low t statistics are generally adjacent with low t statistics (blue).

If our data were truly independent, we would see something that looked more like this.

In [ ]:
base_ts=np.copy(result_random_effects[2])
rand_ts=np.random.shuffle(base_ts)

vas_rgb_vt=cortex.Vertex(base_ts, subject='hcp_999999', cmap='BuWtRd',vmin=-2,vmax=2) 

cortex.quickshow(vas_rgb_vt, with_labels=False, with_rois=False,with_curvature=True);
plt.title("Group level results")

Something that we know about fMRI activations is that they are spatially correlated. We tend to see 'clusters' of activations rather than really dotty t statistics like this. In other words, if we know that a certain voxel is signficant in a certain test, it is quite likely that the voxels directly next to it are also significant. The same applies to non-significant voxels. 

&nbsp;

Hence, we are not really conducting 118584 independent tests - but some number lower than that that is related to number of these clusters. Thus, each of the aforementioned multiple comparisons corrections have the disadvantage of not fully incorporating the correlation structure of the data, namely that points close to one another tend to be correlated. However, by defining the adjacency (or “neighbor”) structure in our data, we can use a **cluster-based** approach to compensate.

Cluster based approaches all start with defining **clusters**, which are contiguous (connecting) regions of the brain that all exceed a cetain statistic.

&nbsp;

<img src = 'https://i.imgur.com/xZfkKoH.png' width = "500" height = "" >

&nbsp;

Here we see the typical effect of different levels of thresholding on 
cluster definitions ; the resulting clusters are randomly color-coded to show
which voxels belong to each cluster. At the lowest threshold, there is one large cluster that
encompasses much of the brain, whereas higher thresholds break up this cluster, at the
expense of excluding many regions that do not survive the higher threshold.

&nbsp;

Given that we know that clusters are a meaningful unit in fMRI analysis, perhaps we can reframe our orginal question:

&nbsp;

"what was the chance of finding a t value this large for the voxel if the null hypothesis was true?"

&nbsp;

And instead, we can ask a different question, which is:

&nbsp;

*"What was the chance of finding a **cluster of this size** if the null hypothesis was true?"*

&nbsp;

This thus allows us to take into account not just the raw t-statistics, **but the sum of t statistics within a cluster**. In doing this,we don't really base everything soley on raw t-statistics, but more about the *spatial extent* of these statistics - thereby respecting the correlation structure of the data.

&nbsp;

By extentsion, we effectively penalize small clusters with few connecting voxels (these tend to be noise anyway), and reward clusters that are larger. 


There are in fact a number of ways of performing these cluster based corrections. I am going to focus on just one of these.


### Permutation-based cluster thresholding

In permutation-based cluster thresholding, we we leverage the [principle of exchangeability](https://www.ohbmbrainmappingblog.com/blog/a-brief-overview-of-permutation-testing-with-examples).

In other words, if the null hypothesis is true, the sign of our ($c\hat{\beta}$) (positive or negative) becomes arbitrary - and we can flip it without changing the distribution of the test statistic. 

Therefore, we can construct a null-distribution of statistics by taking random permutation of subjects, flipping the sign of the ($c\hat{\beta}$), and recording the value of some statistic we compute from this permuted datset.

&nbsp;

Thus, in this case, for each of these random permutation, we would:

1. Compute the t value for each voxel individually.

2. Threshold the t values to define significant ones.

3. Define clusters (adjacent voxels that exceed this threshold).

4. Retain the the maximum sum of t-values within a cluster to build the null distribution.


The p-value is simply the proportion of this null distribution of summed t values that are smaller than those we find in the clusters in the observed data itself.

&nbsp;


**Do not break your head on this too much**. All you really need to know about this procedure is that it evaluates the probability of obtaining a cluster of a given size under randomized conditions that model the null hypothesis.

Ok, lets do this now. I'm only going to do this for one hemisphere to save time

In [ ]:
surfaces=[cortex.polyutils.Surface(*d)
                         for d in cortex.db.get_surf('hcp_999999', 'fiducial')]

from mne.stats import permutation_cluster_1samp_test

t_clust, clusters, p_values, H0 = permutation_cluster_1samp_test(X=betas_to_y_dagger[:,:59292], n_jobs=None, threshold=2.132, adjacency=surfaces[0].adj, out_type='mask',seed=1234)

Note that the first thing we notice printed here is that there are 1403 clusters in our data. In other words, there are 1403 regions with at least 2 significant voxels adjoining one another.

Here, we see the maximum sum of t statistics that we get for each random permutation. This is essentially our 'null distribution'. In order for us to reject the null hypothesis, we should have clusters in our observed data that mostly exceed these values.

In [ ]:
plt.hist(H0)

So based on this null distribution, what kind of p value do we get for each one of our clusters? Let's see and sort them by lowest first. 

In [ ]:
p_values[np.argsort(p_values)][:10]

Oof, we have a few that are very close to .05.

Lets take a look at one cluster with a p value of .0625 now.

In [ ]:
data_to_plot=cortex.Vertex2D(result_random_effects[2],clusters[np.argsort(p_values)[0]],vmin=2,vmax=5,vmin2=0,vmax2=1,subject='hcp_999999',cmap='plasma_alpha')
flatmap=cortex.quickshow(data_to_plot,with_curvature=True,with_rois=False)

This is a pretty big custer, and we get a pretty big sum of t statistics within it.

In [ ]:
np.sum(t_clust[clusters[np.argsort(p_values)[0]]])

The only problem is that we actually got a sum of t statistics larger than this for 1/16 of our permuted samples (red dot on the left is slightly higher than the blue line).

In [ ]:
plt.plot(H0,'r*',label='permuted stat')
plt.ylabel("Max summed t statistic for a cluster")
plt.xlabel("Permutation number")

plt.axhline(y=10590.664518759813,linewidth=1, color='b',label='observed value for cluster')

plt.ylim([0,11000])

And 1/6 equals our p value of 0.0625. The chances of finding a cluster of this size under the null hypothesis are still > .05 Hence, we still technically have not found a significant effect...

What next?

<img src = 'https://dragonflytraining.files.wordpress.com/2013/10/man-with-question-01.png?w=600&h=600' width = "100" height = "" >

**QUESTION:** What possible problems can you see with cluster-based corrections?



# Take a break 4.

Ok, lets take a break and discuss what we have done so far.

<img src = 'https://upload.wikimedia.org/wikipedia/commons/8/81/Stop_sign.png' width = "200" height = "" >

# Region of interest analyses.

By now you may start to suspect that conducting an experiment with 5 participants was doomed to fail with respect to detecting a significant effect.

However, this might be because we are asking a very unspecific, untargetted question about all of the voxels in the brain - which requires us to correct for the number of tests we are performing. This is punitive to the extent that we cannot detect any effects at the group level.

One alternative way to deal with the multiple comparisons problem is a more targetted, confirmatory approach: region-of-interest (ROI) analysis.

## What is an ROI?

An ROI is a pre-defined are of the brain that is thought to be relevant to our effects under study. These can be anatomically or functionally defined. *Anatomical ROIs* are regions of interest based on anatomical landmarks (such as folding pattern, myelination etc), this might include the caudate nucleus, Heschl's gyrus, and anterior cingulate cortex.

Other ROIs are defined based on *functional* properties. Sometimes, the data used for defining such a functional ROI comes from a different run of the same experiment, consisting of a “functional localizer” to define an ROI for focusing the analysis of other data. For example, [floc](https://github.com/VPNL/fLoc) is a procedure for localising face-selective regions of the brain such as the fusiform area (FFA) and occipital face area (OFA) based on contrasting each voxels activity to faces versus other categories of visual stimuli (words, houses etc) and thresholding the resulting statistics. 

&nbsp;

<img src = 'https://ars.els-cdn.com/content/image/3-s2.0-B9780123970251001585-f00158-01-9780123970251.jpg' width = "500" height = "" >

&nbsp;

A slightly different example is that the boundary between the primary visual cortex (V1) and secondary visual cortex (V2) is defined by its selectivity for different parts of the visual field - which can be revealed via a technique called [population receptive field modeling](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3073038/).

The color wheel on the bottom right of the below figure represents the portion of the visual field that each portion of visual cortex is tuned to (expressed as polar angle). We can see that V1 smoothly represents a quadrant of the visual field (blue to red & blue to green - going from horizontal to vertical meridians) - but this reverses at the boundary of V2 (red to blue & green to blue going from vertical to horizontal meridians). This reversal in the visual field representation indicates the boundary between two distinct maps of the visual field - allowing us to parse out V1 and V2.

&nbsp;

<img src = 'https://www.researchgate.net/publication/239463768/figure/fig5/AS:667778742771716@1536222267296/Comparison-of-the-polar-angle-retinotopy-in-human-visual-cortex-relative-to-that.png' width = "500" height = "" >

&nbsp;

In addition, there are many standardizes *atlases* or *parcellations* of the brain that exist from whcih we can define ROIs. Some of these are summarised [here](https://www.lead-dbs.org/helpsupport/knowledge-base/atlasesresources/cortical-atlas-parcellations-mni-space/). 

&nbsp;

One of the most extensive of these is the [Glasser 2016](https://www.nature.com/articles/nature18933) multimodal parcellation of over 180 cortical regions. 

&nbsp;

<img src = 'https://media.springernature.com/lw685/springer-static/image/art%3A10.1038%2Fnature18933/MediaObjects/41586_2016_Article_BFnature18933_Fig3_HTML.jpg?as=webp' width = "500" height = "" >

&nbsp;

This is a truly mind-boggling atlas. It leveraged data from task-based functional contrasts, anatomical cues (e.g. cortical folding patterns, thickness) and cytoarchitecture (myelin etc) to define these regions (hence the word 'multimodal'). You can interact with this atlas using this [nice tool](http://corticalexplorer.com/). Take some time to play around with this.

I have downloaded this atlas, so we also have this at our disposal here: 




In [ ]:
mm=MMP_masker('/content/data/HCP-MMP')
atlas=np.concatenate([mm.lh_labels,mm.rh_labels])

vas_rgb_v=cortex.Vertex(atlas, subject='hcp_999999', cmap='gist_ncar',vmin=np.nanmin(atlas),vmax=np.nanmax(atlas)) 
cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=False);

Here, each color represents a different region of the brain.

We can make 'masks' based on regions defined in this atlas. Here for instance, the fusiform face area (here referred to as the FFC - fusiform face complex is shown in white.

We can use masks like this to limit the scope of our analyses - restricting our analyses to voxels within the mask.

In [ ]:
FFC_mask=mm.make_composite_mask(['FFC'])

vas_rgb_v=cortex.Vertex(FFC_mask[-1], subject='hcp_999999', cmap='gist_ncar',vmin=np.nanmin(FFC_mask[-1]),vmax=np.nanmax(FFC_mask[-1])) 
cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=False);

## Choosing an ROI.

To define an ROI in which to perform a targetted analysis, we would usually consult previous literature to identify structures that could be plausibly related to the effect we are interested in.

&nbsp;

To this end, one tool I find valuable is [neurosynth](https://neurosynth.org/analyses/terms/) Neurosynth is a tool that, amongst other things, can perform “automatic” meta-analyses of fMRI data. As stated on its website, “it takes thousands of published articles reporting the results of fMRI studies, chews on them for a bit, and then spits out images”.

&nbsp;

Take a look at this result [here](https://neurosynth.org/analyses/terms/body/) where I enter the search term 'body'. It reveals z statistics of an “association test” of your selected term across voxels in MNI space, which tests whether voxels are preferentially related to your selected term. In other words, here we have performed a sort of meta-analytic localiser on regions of the brain that are selective to bodies.

&nbsp;

If we navigate to the peak z statistic (-50,-76,-4), we can then click *'whats here'* and then on the *'associations'* tab to see what other search terms load heavily onto this location.

&nbsp;

<img src = 'https://i.imgur.com/1K16Bbu.png' width = "200" height = "" >

&nbsp;

<img src = 'https://i.imgur.com/ch7W1c8.png' width = "600" height = "" >

&nbsp;

We see lots of interesting things here. One is 'action observation'- which makes sense given that when we view bodies we usually view them performing actions. Furthermore, we also see labels for different brain regions. One of these is 'V5' and another is 'MT'. Actually, these labels are often [used interchangeably](https://pubmed.ncbi.nlm.nih.gov/15702885/) to refer to the human motion-sensitive visual cortex - which is also why we see the term 'motion' in here.

&nbsp;


Thus, we have good reason to suspect, based on previous literature that area MT will be selective to bodies v scenes. Hence, we can now re-pitch our research question:

&nbsp;

"*Is there a greater average response to bodies than scenes in human area MT?*". 

&nbsp;

On the one hand this seems like a bit of a crafty thing to do - side-stepping the multiple comparisons problem like this. However - think of it this way: We can be fairly certain that only a small portion of cortex is going to show body-selectivity. Why then, would we punish ourselves by fishing around the entire brain and suffering all the penalties that we incur for doing this?

&nbsp;

For instance, earlier on we looked at what we can be fairly certain are the EBA and the FBA. Given the consistency of these regions across subjects and the scope of our question - which is about selectivity to bodies - not a comprehensive model of brain-wide function - its kind of wild to hamstring ourselves with the whole-brain multiple-comparisons problem when we have a good reason for being more specific.

&nbsp;

One interesting factoid for you. It turns out that what we call 'MT' and 'EBA' heavily overlap with one another - to the extent that different groups of researchers are probably using these labels to refer to the same region of the brain.

<img src = 'https://i.imgur.com/uXZrkqV.png' width = "400" height = "" >

The region drawn in white here is the EBA, which I defined based on an independent functional contrast between bodies anod other categories of stimulus, using the floc localizer. In pink you can see the location of MT, as estimated by the Glasser parcelation. This just goes to show that these labels we construct for different brain regions are heavily dependent on the data we use to define them:

&nbsp;

"*It shows a large response to bodies? Let's call it the body area!*"

"*It shows a large response to visual motion patterns? Let's call it motion sensitive cortex!*".

&nbsp;

 It also illustrates that mapping one function to one brain area is probably not a good idea.......


&nbsp;

Regardless, luckily for us, as you can see [here](https://docs.google.com/spreadsheets/d/1hZYPbEGX_MmsaajTdOcbiI8DxghrtZCT/edit?usp=sharing&ouid=115381703141154937331&rtpof=true&sd=true) area 'MT' (row 25) is a region that is defined in our Glasser multimodal atlas. Therefore, we have everything we need to perform an ROI analysis on MT.




## Setting up $y^{\dagger}$ for roi analysis.

The first thing we want to is limit our $c\hat{\beta}$ to those within our ROI, to this end we define a mask.


In [ ]:
MT_mask=mm.make_composite_mask(['MT'])

vas_rgb_v=cortex.Vertex(MT_mask[-1], subject='hcp_999999', cmap='gist_ncar',vmin=np.nanmin(MT_mask[-1]),vmax=np.nanmax(MT_mask[-1])) 
cortex.quickshow(vas_rgb_v, with_labels=False, with_rois=False,with_curvature=False);

As you can see - we see got some consistently high voxel-wise t statistics within this region. This does seem like an ROI that is a good candidate for detecting a significant effect.

In [ ]:
data_to_plot=cortex.Vertex2D(result_random_effects[2],MT_mask[-1],vmin=2,vmax=5,vmin2=0,vmax2=1,subject='hcp_999999',cmap='plasma_alpha')
flatmap=cortex.quickshow(data_to_plot,with_curvature=True,with_rois=False)

zoomrect=[-140,140,-30,40]

zoom_to_rect(zoomrect)

We then take the mean ($c\hat{\beta}$) across voxels in this mask for each subject. This gives us 5 values, each reflecting the mean ($c\hat{\beta}$) within MT for each subject

In [ ]:
Y_dagger_MT=np.mean(betas_to_y_dagger[:,FFC_mask[-1]],axis=1)

In [ ]:
Y_dagger_MT.shape

## Fitting the model

The model fitting is no different. We fit the random effects model with these voxel-averaged MT ($c\hat{\beta}$) as our $y^{\dagger}$ 

In [ ]:
result_random_effects_MT=compute_contrast(X_dagger,Y_dagger_MT.T,[1])

Here, we get a massive t value and p value well less than .05

In [ ]:
result_random_effects_MT[2],1 - t_dist.cdf(result_random_effects_MT[2])

In [ ]:
plt.scatter(np.ones(5), Y_dagger_MT, s=120, facecolors='none', edgecolors='r')
plt.ylabel("Single subject effect size for MT")
plt.axhline(y=0,c='k')
plt.axhline(y=result_random_effects_MT[0],c='r',label='group mean (cope)')
plt.errorbar(x=1, y=result_random_effects_MT[0], yerr=result_random_effects_MT[1], markersize=8,fmt='o',label='variance (varcope)',uplims=True, lolims=True)
plt.legend()
plt.ylim([-0.25,1.75])

Since we have asked this very specific, targetted question in a confirmatory way we no longer suffer from the multiple comparisons problem. We can therefore reject our null hypotheses and say "Images of bodies have more influence on the average response of MT than scenes in the general population".

# Summary
## Great work!!!

We covered a lot of ground here! 👏

1. Fixed effects modeling to combine runs of data.
2. Random effects modeling to make inferences about the general population.
3. Different methods for dealing with the multiple comparisons problem.
4. ROI-based analyses - leveraging existing atlases and online meta-analytic tools.

&nbsp;

---

&nbsp;

### Some key points

1. Second-level analysis operates via the same principles as first-level analysis. We use exactly the same formulae and estimate the parameters in exactly the same way. The only differece is that the variable we are now trying to explain is the parameters estimated at the single-subject level.
2. There are multiple methods for correcting for the multiple comparisons problem - all of these have various strengths, weakness and complexities.
3. There is a great strength in specific, targetted hypotheses - which keep us from a voxel-wise "fishing expedition" and all the problems that incurs. 

# Things to think about

1. Would you expect our p values to survive corrections if we had double the participants?
2. What effect do you think spatial smoothing might have on our cluster analyses?
3. Take a browse of neurosynth and search for 'visual word'. Think of some interesting ROI-based analyses we could perform based on the design matrix we have for our movie. 


